In [0]:
import pickle as pkl
import time
import os
import numpy as np
import sys
from music21 import instrument, note, stream, chord, duration

import matplotlib.pyplot as plt
from tensorflow.keras.layers import LSTM, Input, Dropout, Dense, Activation, Embedding, Concatenate, Reshape, Bidirectional
from tensorflow.keras.layers import Flatten, RepeatVector, Permute, TimeDistributed
from tensorflow.keras.layers import Multiply, Lambda, Softmax
import tensorflow.keras.backend as K 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop



In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#sys.path.append('/content/gdrive/My Drive/cs230')
project_folder = "/content/gdrive/My Drive/cs230"
#data_folder = project_folder + "/data"
#jazz_folder = project_folder + "/data/sumuzhao/Jazz"
classical_folder = project_folder + "/data/sumuzhao/Classic"

In [0]:
section = 'compose'
run_id = '001'
music_name = 'Study_No_1_Opus_105'
run_folder = project_folder + '/run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])

# model params
embed_size = 100
rnn_units = 256

In [0]:
store_folder = os.path.join(run_folder, 'store')

with open(os.path.join(store_folder, 'distincts'), 'rb') as filepath:
    distincts = pkl.load(filepath)
    note_names, n_notes, duration_names, n_durations = distincts

with open(os.path.join(store_folder, 'lookups'), 'rb') as filepath:
    lookups = pkl.load(filepath)
    note_to_int, int_to_note, duration_to_int, int_to_duration = lookups

In [0]:
len(note_names), n_notes, len(duration_names), n_durations

(456, 456, 19, 19)

In [0]:
def create_network(n_notes, n_durations, embed_size = 100, rnn_units = 256):

    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))

    x1 = Embedding(n_notes, embed_size)(notes_in)
    x2 = Embedding(n_durations, embed_size)(durations_in) 

    x = Concatenate()([x1,x2])
    x = Bidirectional(LSTM(rnn_units, return_sequences=True))(x)
    x = LSTM(rnn_units, return_sequences=True)(x)

    e = Dense(1, activation='tanh')(x)
    e = Reshape([-1])(e)
    alpha = Activation('softmax')(e)
    alpha_repeated = Permute([2, 1])(RepeatVector(rnn_units)(alpha))

    c = Multiply()([x, alpha_repeated])
    c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)
                                    
    notes_out = Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
    durations_out = Dense(n_durations, activation = 'softmax', name = 'duration')(c)
   
    model = Model([notes_in, durations_in], [notes_out, durations_out])

    opti = RMSprop(lr = 0.001)
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opti)

    return model

In [22]:
weights_folder = os.path.join(run_folder, 'weights')
weights_file = 'weights.h5'

model = create_network(n_notes, n_durations, embed_size, rnn_units)

# Load the weights to each node
weight_source = os.path.join(weights_folder,weights_file)
model.load_weights(weight_source)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    21200       input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 100)    600         input_4[0][0]                    
____________________________________________________________________________________________

In [0]:
def sample_with_temp(preds, temperature):

    if temperature == 0:
        return np.argmax(preds)
    else:
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        return np.random.choice(len(preds), p=preds)

In [0]:
# prediction params
notes_temp=0.5
duration_temp = 0.
max_extra_notes = 32
max_seq_len = 32
seq_len = 32

def load_notes_durations():
# load notes of all the training files
  with open(os.path.join(store_folder, 'notes'), 'rb') as f:
    notes = pkl.load(f)
  
  with open(os.path.join(store_folder, 'durations'), 'rb') as f:
    durations = pkl.load(f)

  return notes, durations
# notes = ['START']
# durations = [0]

# if seq_len is not None:
#     notes = ['START'] * (seq_len - len(notes)) + notes
#     durations = [0] * (seq_len - len(durations)) + durations

all_notes, all_durations = load_notes_durations()

In [0]:
def starting_input():
  index = np.random.randint(0, len(all_notes) - 32)
  notes = all_notes[index: index + 32]
  durations = all_durations[index: index + 32]

  notes_input_sequence, durations_input_sequence = [], []
    
  for n, d in zip(notes,durations):
    note_int = note_to_int[n]
    duration_int = duration_to_int[d]
    
    notes_input_sequence.append(note_int)
    durations_input_sequence.append(duration_int)
    
    prediction_output.append([n, d])
    
      # if n != 'START':
      #   midi_note = note.Note(n)

      #   new_note = np.zeros(128)
      #   new_note[midi_note.pitch.midi] = 1
      #   #overall_preds.append(new_note)

    return notes_input_sequence, durations_input_sequence

In [0]:
def section(notes_input_sequence, durations_input_sequence):
  section_output = []

  for note_index in range(max_extra_notes):
    prediction_input = [
        np.array([notes_input_sequence])
        , np.array([durations_input_sequence])
       ]

    notes_prediction, durations_prediction = model.predict(prediction_input, verbose=0)
    
    new_note = np.zeros(128)
    
    for idx, n_i in enumerate(notes_prediction[0]):
        try:
            note_name = int_to_note[idx]
            midi_note = note.Note(note_name)
            new_note[midi_note.pitch.midi] = n_i
            
        except:
            pass
            
    note_sample = sample_with_temp(notes_prediction[0], notes_temp)
    duration_sample = sample_with_temp(durations_prediction[0], duration_temp)

    note_result = int_to_note[note_sample]
    duration_result = int_to_duration[duration_sample]
    
    section_output.append([note_result, duration_result])

    notes_input_sequence.append(note_sample)
    durations_input_sequence.append(duration_sample)
    
    if len(notes_input_sequence) > max_seq_len:
        notes_input_sequence = notes_input_sequence[1:]
        durations_input_sequence = durations_input_sequence[1:]
  
#     print(note_result)
#     print(duration_result)
    variation_seed = [notes_input_sequence[-32:], durations_input_sequence[-32:]]
    if note_result == 'START':
        # return notes_input_sequence, durations_input_sequence, prediction_output, variation_seed
        return section_output, variation_seed
  # return notes_input_sequence, durations_input_sequence, prediction_output, variation_seed
  return section_output, variation_seed
#print('Generated sequence of {} notes'.format(len(prediction_output)))
  

In [0]:
structure = ['A', 'B', 'A']
prediction_output = []
seed_dict = {}
notes_input_sequence, durations_input_sequence = starting_input()

for s in structure:
  if s in seed_dict:
    # notes_input_sequence, durations_input_sequence, section_output, variation_seed = section(variation_seed[0], variation_seed[1])
    section_output, variation_seed = section(seed_dict[s][0], seed_dict[s][1])
  else:
    # notes_input_sequence, durations_input_sequence, section_output, variation_seed = section(notes_input_sequence, durations_input_sequence)
    section_output, variation_seed = section(notes_input_sequence, durations_input_sequence)

  prediction_output += section_output  
  seed_dict[s] = variation_seed

In [33]:
output_folder = os.path.join(run_folder, 'output')

midi_stream = stream.Stream()

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    note_pattern, duration_pattern = pattern
    # pattern is a chord
    if ('.' in note_pattern):
        notes_in_chord = note_pattern.split('.')
        chord_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(current_note)
            new_note.duration = duration.Duration(duration_pattern)
            new_note.storedInstrument = instrument.Piano()
            chord_notes.append(new_note)
        new_chord = chord.Chord(chord_notes)
        midi_stream.append(new_chord)
    elif note_pattern == 'rest':
    # pattern is a rest
        new_note = note.Rest()
        new_note.duration = duration.Duration(duration_pattern)
        new_note.storedInstrument = instrument.Piano()
        midi_stream.append(new_note)
    elif note_pattern != 'START':
    # pattern is a note
        new_note = note.Note(note_pattern)
        new_note.duration = duration.Duration(duration_pattern)
        new_note.storedInstrument = instrument.Piano()
        midi_stream.append(new_note)



midi_stream = midi_stream.chordify()
timestr = time.strftime("%Y%m%d-%H%M%S")
midi_stream.write('midi', fp=os.path.join(output_folder, 'output-' + timestr + '.mid'))

'/content/gdrive/My Drive/cs230/run/compose/001_Study_No_1_Opus_105/output/output-20200601-024848.mid'

In [34]:
prediction_output

[['C2.C3.G4.G5', 0.25],
 ['F4', 0.25],
 ['G3.C4.E4.E5', 0.25],
 ['C5.C6', 0.25],
 ['C5.C6', 0.25],
 ['A3.C4.F4', 0.25],
 ['E3.G3.C4.G4', 0.25],
 ['C2.C3.E5.E6', 0.25],
 ['G2.E4.C5', 0.25],
 ['E3.G3.C4.G4', 0.25],
 ['G1.G2.D4.D5', 0.25],
 ['G1.G2.D4.D5', 0.25],
 ['G1.G2.D4.D5', 0.25],
 ['A3.C4.F4', 0.25],
 ['C2.C3.C4.C5', 0.25],
 ['F3.G3.B3.D6', 0.25],
 ['E3.G3.C4.G4', 0.25],
 ['E3.G3.C4.G4', 0.25],
 ['E3.G3.C4.G4', 0.25],
 ['E3.G3.C4.G4', 0.25],
 ['C3.G3.C4.E4.G4.C5.E5', 0.25],
 ['G3.C4.E4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4.F#5', 0.25],
 ['D3.F#3.C4.A5', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.F#3.C4', 0.25],
 ['D3.G3.B3.D5', 0.25],
 ['D3.G3.B3.D5', 0.25],
 ['D3.G3.B3.G5', 0.25],
 ['D3.G3.B3.B5', 0.25],
 ['D3.G3.B3.B5', 0.25],
 ['D3.G3.B3.B5', 